In [57]:
import pandas as pd
import re

##### 0. ITEMID

In [145]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ITEMS.csv")
df

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12482,14518,226757,GCSMotorApacheIIValue,GCSMotorApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12483,14519,226758,GCSVerbalApacheIIValue,GCSVerbalApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12484,14520,226759,HCO3ApacheIIValue,HCO3ApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
12485,14521,226760,HCO3Score,HCO3Score,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN


In [146]:
# print(list(df['CATEGORY'].unique()))

In [147]:
# print(list(df['LABEL'].unique()))

In [148]:
# 1. ARDS 논문의 12개 feature + ChatGPT에게 물어본 General한 LABEL
L_list = ['WBC', 'White Blood', 'Albumin', 'Platelet', 'pH', 'Lactate Level', 'Fi02', 'Pa02', 'Heart Rate', 'temperature', 'BMI', 'Body Mass Index',
          'PaCO2', 'Base Excess', 'Hemoglobin', 'Hematocrit', 'Sodium', 'Potassium', 'Chloride', 'Bicarbonate', 'Blood Urea Nitrogen', 'BUN', 'Creatinine', 
          'Glucose', 'Calcium', 'Magnesium', 'Phosphate', 'Bilirubin', 'Aspartate Aminotransferase', 'AST', 'Alanine Aminotransferase',
          'ALT','Alkaline', 'Phosphatase' ,'ALP','Protein']
L_list = [s.lower().replace(' ', '') for s in L_list]

df['LABEL'] = df['LABEL'].str.lower() #소문자로 변경
df['LABEL'] = df['LABEL'].str.replace(' ', '', regex=True).str.strip()#공백 제거
pattern = '|'.join(L_list)
df1 = df[pd.notna(df['LABEL']) & df['LABEL'].str.contains(pattern)]
# df1 = df[df['LABEL'].str.contains(pattern)]
# df1 = df[df['LABEL'].isin([x.lower() for x in L_list])]
ID1 = df1["ITEMID"].tolist()
print("Respiratory 관련 LABEL 개수: ", len(ID1))

# 2. Category가 Respiratory인 ITEMID 
df['CATEGORY'] = df['CATEGORY'].str.lower() #소문자로 변경
df2 = df[df['CATEGORY']==""]
ID2 = df2["ITEMID"].tolist()
print("Category가 Respiratory인 ITEMID 개수", len(ID2))

# 3. ID1, ID2 합치기
ID_list = ID1+ID2
print("전체 ITEMID 개수: ", len(ID_list))

# 4. ID_list에 해당한는 data 확인
df = df[df["ITEMID"].isin(ID_list)]
df.drop(columns=["ROW_ID", "ABBREVIATION", "DBSOURCE", "LINKSTO"], inplace=True)
df

Respiratory 관련 LABEL 개수:  1063
Category가 Respiratory인 ITEMID 개수 0
전체 ITEMID 개수:  1063


/tmp/ipykernel_1751057/3690003865.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["ROW_ID", "ABBREVIATION", "DBSOURCE", "LINKSTO"], inplace=True)


,ITEMID,LABEL,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
47,1042,acetomenophen,NaN,NaN,NaN,NaN
119,1126,art.ph,abg,NaN,NaN,NaN
120,1127,"wbc(4-11,000)",hematology,NaN,NaN,NaN
125,1162,bun,NaN,NaN,NaN,NaN
132,1192,pericardialpressure,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
12471,226746,apacheiichronichealth,scores - apache ii,NaN,Text,NaN
12472,226747,apacheii-chronichealthpoints,scores - apache ii,NaN,Text,NaN
12476,226751,creatinineapacheiiscore,scores - apache ii,None,Numeric,NaN
12477,226752,creatinineapacheiivalue,scores - apache ii,None,Numeric,NaN


In [149]:
df[['LABEL']].tail(20)

,LABEL
12256,zgastric/tfresidualintake
12262,orplateletintake
12272,guirrigant-amphotericinb
12285,esophogealballoon
12296,pacugastric
12302,lastmenses
12326,photodynamictherapypdt(bronch)
12365,healthcareproxy
12366,temperaturefahrenheit
12367,temperaturecelsius


##### 1. LABEVENTS

In [104]:
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")
df1

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [105]:
df1 = df1[df1["ITEMID"].isin(ID_list)]
df1

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG


##### 2. OUTPUTEVENT

In [106]:
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/OUTPUTEVENTS.csv")
df2

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,4340476,68375,123645.0,276251.0,2175-09-05 23:00:00,226588,30.0,mL,2175-09-05 22:48:00,16915,NaN,NaN,NaN
4349214,4340477,68375,123645.0,276251.0,2175-09-06 00:00:00,226588,0.0,mL,2175-09-06 00:03:00,20971,NaN,NaN,NaN
4349215,4340478,68375,123645.0,276251.0,2175-09-06 01:00:00,226588,40.0,mL,2175-09-06 01:13:00,20971,NaN,NaN,NaN
4349216,4340479,68375,123645.0,276251.0,2175-09-06 02:00:00,226588,20.0,mL,2175-09-06 02:14:00,20971,NaN,NaN,NaN


In [107]:
df2 = df2[df2["ITEMID"].isin(ID_list)]
df2

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
386,200,21219,177991.0,225765.0,2142-09-06 20:00:00,40052,0.0,NaN,2142-09-06 20:00:00,21570,NaN,NaN,NaN
387,201,21271,140427.0,271112.0,2187-08-09 00:00:00,40059,250.0,ml,2187-08-09 01:16:00,15023,NaN,NaN,NaN
388,202,21271,140427.0,271112.0,2187-08-10 00:00:00,40059,600.0,ml,2187-08-10 00:18:00,15023,NaN,NaN,NaN
389,203,21271,140427.0,271112.0,2187-08-10 06:00:00,40059,100.0,ml,2187-08-10 06:04:00,15023,NaN,NaN,NaN
390,204,21271,140427.0,271112.0,2187-08-10 17:00:00,40059,400.0,ml,2187-08-10 17:05:00,15823,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348469,4348361,76841,173944.0,259862.0,2123-08-20 05:09:00,226575,25.0,mL,2123-08-20 05:10:00,17461,NaN,NaN,NaN
4348470,4348362,76841,173944.0,259862.0,2123-08-20 21:15:00,226575,100.0,mL,2123-08-20 21:15:00,18804,NaN,NaN,NaN
4348794,4344770,84095,131026.0,269064.0,2164-08-24 15:14:00,226576,25.0,mL,2164-08-24 15:15:00,14988,NaN,NaN,NaN
4348795,4344771,84095,131026.0,269064.0,2164-08-24 16:00:00,226576,25.0,mL,2164-08-24 17:03:00,14988,NaN,NaN,NaN


##### 3. MICROBIOLOGYEVENTS

In [117]:
df3 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/MICROBIOLOGYEVENTS.csv")
df3

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631721,630931,99783,126090,2125-11-05 00:00:00,2125-11-05 13:15:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,2.0,NaN,NaN,NaN,NaN,NaN,NaN
631722,630932,99783,126090,2125-11-06 00:00:00,2125-11-06 10:24:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
631723,630933,99783,126090,2125-11-06 00:00:00,2125-11-06 10:24:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,2.0,NaN,NaN,NaN,NaN,NaN,NaN
631724,630934,99783,126090,2125-11-07 00:00:00,2125-11-07 12:40:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
ID_list = [float(x) for x in ID_list]
df3 = df3[df3[["SPEC_ITEMID", "ORG_ITEMID", "AB_ITEMID"]].isin(ID_list).any(axis=1)]
df3

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
14,758,101,175533,2196-09-28 00:00:00,2196-09-28 20:58:00,70079.0,URINE,80155.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",1.0,NaN,NaN,NaN,NaN,NaN,NaN
27,771,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90012.0,GENTAMICIN,<=0.5,<=,1.0,S
28,772,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90016.0,OXACILLIN,0.5,=,1.0,S
29,773,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90006.0,ERYTHROMYCIN,=>8,=>,8.0,R
30,774,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90002.0,PENICILLIN,=>0.5,=>,1.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631681,630891,99776,136231,2171-07-18 00:00:00,2171-07-18 12:40:00,70076.0,TISSUE,80023.0,STAPH AUREUS COAG +,1.0,90025.0,LEVOFLOXACIN,<=0.12,<=,0.0,S
631682,630892,99776,136231,2171-07-18 00:00:00,2171-07-18 12:40:00,70076.0,TISSUE,80155.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",1.0,NaN,NaN,NaN,NaN,NaN,NaN
631683,630893,99776,136231,2171-07-18 00:00:00,2171-07-18 12:40:00,70076.0,TISSUE,80023.0,STAPH AUREUS COAG +,1.0,90012.0,GENTAMICIN,<=0.5,<=,1.0,S
631684,630894,99776,136231,2171-07-18 00:00:00,2171-07-18 12:40:00,70076.0,TISSUE,80023.0,STAPH AUREUS COAG +,1.0,90016.0,OXACILLIN,0.5,=,1.0,S
